In [1]:
!pip install openai
!pip install langchain
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.2 MB/s eta 0:00:00


In [6]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key_team2')

In [7]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

In [9]:
#대화형 프롬프트 생성
from langchain.prompts.chat import ChatPromptTemplate

#모델 출력 >> 구문 분석 >> 원하는 형식으로 변환
from langchain.schema import BaseOutputParser

# 댓글의 감성을 정확하게 분류하는 분류기 역할을 합니다.
# 사용자가 입력한 설명에 대해 해당 설명이 긍정적이면 '1'을 쓰고, 부정적이면 '0'을 씁니다.
# 다른 텍스트 없이 정수 '1' 또는 '0'만 출력합니다
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Act as a classifier that accurately categorizes the sentiment of comments. Given a user-input comment, write '1' if the comment is positive, and '0' if the comment is negative. Output the integer '1' or '0' only, without any other text."),
    ("human", "{input_text}")

])

class classificationOutputParser(BaseOutputParser):
    def parse(self, text: str):
        if '1' in text:
            return 1
        return 0

#Langchain Expression Language(LCEL)
# >> 사용자 입력에 대해 대화 템플릿 사용
# >> 모델에게 요청 (request)
# >> 모델의 출력을 분류하여 반환

chain = chat_prompt_template | chat_model | classificationOutputParser()

def langchain_llm(input_text):
    output = chain.invoke({'input_text' : input_text})
    return output

def classify_text(input_text):
    output = langchain_llm(input_text)
    return output

분석에 사용할 데이터 로드 >> 분석

In [10]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x7c3e9bdfe2c0>)

In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
df = pd.read_csv('tarr_train.txt', sep='\t')
df.head()

,id,comment,label
0,1,여기 음식은 언제 와도 실망시키지 않아요. 최고!,1
1,2,여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.,1
2,3,"진짜 깔끔하고, 맛도 좋았어요. 추천합니다!",1
3,4,왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음,0
4,5,인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!,1


In [12]:
len(df)

300

In [16]:
actual_labels = []
predicted_labels = []

total = len(df)

for index, row in df.iterrows():
    comment = row['comment']
    actual_label = row['label']

    #llm에 질의 >> 1(긍정) 0(부정) 코멘트 분류
    predicted_label = classify_text(comment)

    #실제 분류 데이터와 예측 분류 데이터 추가
    actual_labels.append(actual_label)
    predicted_labels.append(predicted_label)

    print(f'[{index+1}] / [{total}]')
    print("comment: ", comment)
    print("actual_class: ", actual_label)
    print("predicted_class: ", predicted_label)
    print("--" * 30)

    if index > 20:
        break


[1] / [300]
comment:  여기 음식은 언제 와도 실망시키지 않아요. 최고!
actual_class:  1
predicted_class:  1
------------------------------------------------------------
[2] / [300]
comment:  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
actual_class:  1
predicted_class:  1
------------------------------------------------------------
[3] / [300]
comment:  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
actual_class:  1
predicted_class:  1
------------------------------------------------------------
[4] / [300]
comment:  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
actual_class:  0
predicted_class:  0
------------------------------------------------------------
[5] / [300]
comment:  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
actual_class:  1
predicted_class:  1
------------------------------------------------------------
[6] / [300]
comment:  메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
actual_class:  1
predicted_class:  1
------------------------------------------------------------
[7] / [300]
comment:  사진과 음식이 너무 달라서 실망했습니다.
actual_class:  0
predicted_class:  0
----

In [17]:
#정확도 계산
acc = accuracy_score(actual_labels, predicted_labels)
print(f"정확도: {acc * 100:.2f}%")

정확도: 100.00%


In [18]:
#confustion matrix 계산
cm = confusion_matrix(actual_labels, predicted_labels, labels = [1,0])
cm

array([[14,  0],
       [ 0,  8]])

In [19]:
# Confusion matrix 표현
print("\nConfusion Matrix:")
print("         Predicted:")
print("         긍정    부정")
print("Actual")
print("긍정      {:<5}  {:<5}".format(cm[0][0], cm[0][1]))
print("부정      {:<5}  {:<5}".format(cm[1][0], cm[1][1]))


Confusion Matrix:
         Predicted:
         긍정    부정
Actual
긍정      14     0    
부정      0      8    
